#Perfilamiento de PCA

Angel Rafael Ortega Ramírez 	123972

Elizabeth Rodriguez 	191430

Karla Alfaro Pizaña 	137314

Leonardo Marín 	175903

Mario Rodríguez 	164471

Elizabeth Viveros

In [0]:
%pip install -q --user line_profiler
%pip install -q --user memory_profiler
%pip install -q --user psutil
%pip install -q --user guppy3

In [22]:
##Instalación de /usr/bin/time
%%bash
sudo apt-get install time

Reading package lists...
Building dependency tree...
Reading state information...
time is already the newest version (1.7-25.1build1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [0]:
import math
from scipy.integrate import quad
import time
import os
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import platform
from datetime import datetime

In [3]:
#Montamos nuestro Google Drive con el set de datos
from google.colab import drive
drive.mount('/content/gdrive') #Se da click en el URL que aparece y se ingresa la clave de autorización

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!mkdir '/content/gdrive/My Drive/Proyecto_Final_MNO/' #Se crea la carpeta Proyecto_Final_MNO en la primer ejecución
os.chdir('/content/gdrive/My Drive/Proyecto_Final_MNO')
print("Directorio del Proyecto Final" , os.getcwd())

mkdir: cannot create directory ‘/content/gdrive/My Drive/Proyecto_Final_MNO/’: File exists
Directorio del Proyecto Final /content/gdrive/My Drive/Proyecto_Final_MNO


### Plataforma en la que se realizarán las pruebas**

In [97]:
#Función para escalar cifras en Bytes a las unidades correspondientes
def get_size(bytes, suffix="B"):

    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*40, "Información del Sistema", "="*40)
uname = platform.uname()
print(f"Sistema: {uname.system}")
print(f"Nombre: {uname.node}")
print(f"OS: {uname.release}")
print(f"Versión: {uname.version}")
print(f"Máquina: {uname.machine}")
print(f"Procesador: {uname.processor}")

# Detalles de la memoria
print("="*40, "Memoria", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Desponible: {get_size(svmem.available)}")
print(f"Utilizada: {get_size(svmem.used)}")
print(f"Porcentaje: {svmem.percent}%")
print("="*40, "CPU", "="*40)
# number of cores
print("Cores Físicos:", psutil.cpu_count(logical=False))
print("Cores Totales:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Frecuencia del CPU : {psutil.cpu_freq()}%")
# Disk Information
print("="*40, "Discos Duros", "="*40)
print("Particiones:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Dispositivo: {partition.device} ===")
    print(f"  Montura: {partition.mountpoint}")
    print(f"  Tipo de Sistema de Archivos: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Tamaño Total: {get_size(partition_usage.total)}")
    print(f"  Usado: {get_size(partition_usage.used)}")
    print(f"  Libre: {get_size(partition_usage.free)}")
    print(f"  Porcentaje: {partition_usage.percent}%")

======================================== Información del Sistema ========================================
Sistema: Linux
Nombre: 59992c125a99
OS: 4.19.104+
Versión: #1 SMP Wed Feb 19 05:26:34 PST 2020
Máquina: x86_64
Procesador: x86_64
======================================== Memoria ========================================
Total: 12.72GB
Desponible: 11.74GB
Utilizada: 4.10GB
Porcentaje: 7.7%
======================================== CPU ========================================
Cores Físicos: 1
Cores Totales: 2
Frecuencia del CPU : None%
======================================== Discos Duros ========================================
Particiones:
=== Dispositivo: /dev/sda1 ===
  Montura: /etc/resolv.conf
  Tipo de Sistema de Archivos: ext4
  Tamaño Total: 113.98GB
  Usado: 32.95GB
  Libre: 81.01GB
  Porcentaje: 28.9%
=== Dispositivo: /dev/sda1 ===
  Montura: /etc/hostname
  Tipo de Sistema de Archivos: ext4
  Tamaño Total: 113.98GB
  Usado: 32.95GB
  Libre: 81.01GB
  Porcentaje: 28.9%
=== 

In [0]:
#Carga de Datos
#Se obtiene una vista inicial se los datos
df = pd.read_csv('datos_limpios.csv', nrows = 2000)

In [55]:
# Estructura de los datos
df.shape

(2000, 272)

In [56]:
# Conservar variables de la sección 3
seccion_3 = df.loc[:, ['p10', 'p11', 'p12', 'p13', 'p14']]
seccion_3.head()

,p10,p11,p12,p13,p14
0,2.0,1.0,1.0,2.0,2.0
1,4.0,3.0,3.0,5.0,6.0
2,4.0,1.0,2.0,5.0,5.0
3,2.0,1.0,1.0,3.0,3.0
4,2.0,2.0,3.0,3.0,4.0


## Medición de tiempos

### 1) PCA por usando la SVD (utilizando la SVD de numpy)

In [0]:
# Función para PCA
def PCA_from_SVD(A,num_componentes):
    """
    Función para PCA a partir de la SVD 
    params: A			matriz de datos
            num_componentes 	número de componentes deseados
    return: valores_singulares	Los valores singulares de la descomposición SVD
	    componentes		Los coeficientes para calcular los componentes principales
	    Z			Los datos transformados (componentes principales)
	    varianza_explicada	La varianza explicada por cada componente principal
    """
    
    # Centrar los datos
    A = np.array(A) # convertir los datos a un numpy array por si vienen de un DataFrame
    A_centered = A - A.mean(axis=0)
    
    # Modificar esta línea de código, mandar a llamar la función creada por el equipo 
    # Calcular SVD
    U, S, Vt = np.linalg.svd(A_centered)
    
    # Los valores singulares
    valores_singulares = S
    
    # Los componentes (coeficientes)
    componentes = (np.transpose(Vt))*-1
    
    # Los datos transformados (componentes principales)
    Z = A_centered@np.transpose(Vt)
    
    # La varianza explicada
    varianza_explicada = S**2/np.sum(S**2)
    
    # regresar 4 objetos
    return valores_singulares[:(num_componentes)], componentes[:(num_componentes)], Z[:,:(num_componentes)], varianza_explicada[:(num_componentes)]

In [104]:
# Medición de la función
start_time = time.time()
valores_singulares, coeficientes, Z, varianza_explicada = PCA_from_SVD(seccion_3,2)
end_time = time.time()
ecs = end_time-start_time
print("PCA_from_SVD tomó",secs,"segundos" )

PCA_from_SVD tomó 0.37876248359680176 segundos


In [59]:
valores_singulares

array([92.84675616, 52.8802986 ])

In [60]:
coeficientes

array([[-0.04822396,  0.21919613, -0.95970233,  0.13388036,  0.10331985],
       [-0.3165382 , -0.5410388 , -0.05109762,  0.68676775, -0.36444433]])

In [61]:
varianza_explicada

array([0.60014616, 0.19467553])

### 2) Validación a través de `/usr/bin/time`

In [119]:
# Damos de alta la función como .py
%%file PCA_from_SVD.py
def PCA_from_SVD(A,num_componentes):
    """
    Función para PCA a partir de la SVD 
    params: A			matriz de datos
            num_componentes 	número de componentes deseados
    return: valores_singulares	Los valores singulares de la descomposición SVD
	    componentes		Los coeficientes para calcular los componentes principales
	    Z			Los datos transformados (componentes principales)
	    varianza_explicada	La varianza explicada por cada componente principal
    """
    
    # Centrar los datos
    A = np.array(A) # convertir los datos a un numpy array por si vienen de un DataFrame
    A_centered = A - A.mean(axis=0)
    
    # Modificar esta línea de código, mandar a llamar la función creada por el equipo 
    # Calcular SVD
    U, S, Vt = np.linalg.svd(A_centered)
    
    # Los valores singulares
    valores_singulares = S
    
    # Los componentes (coeficientes)
    componentes = (np.transpose(Vt))*-1
    
    # Los datos transformados (componentes principales)
    Z = A_centered@np.transpose(Vt)
    
    # La varianza explicada
    varianza_explicada = S**2/np.sum(S**2)
    
    return valores_singulares[:(num_componentes)], componentes[:(num_componentes)], Z[:,:(num_componentes)], varianza_explicada[:(num_componentes)]
#if __name__=="__main__": #Conforme a las notas añadimos este bloque para ejecución de la función
#        print("aproximación: {:0.6e}".format(valores_singulares, coeficientes, Z, varianza_explicada = PCA_from_SVD(seccion_3,2)))

Overwriting PCA_from_SVD.py


In [121]:
#real que mide el wall clock o elapsed time.
#user que mide la cantidad de tiempo de tu ejecución que la CPU gastó para funciones que no están relacionadas con el kernel* del sistema.
#sys que mide la cantidad de tiempo de tu ejecución que la CPU gastó en funciones a nivel de kernel del sistema.

%%bash
/usr/bin/time -p python3 PCA_from_SVD.py

real 0.04
user 0.03
sys 0.00


In [122]:
%%bash
/usr/bin/time --verbose python3 PCA_from_SVD.py

	Command being timed: "python3 PCA_from_SVD.py"
	User time (seconds): 0.04
	System time (seconds): 0.02
	Percent of CPU this job got: 96%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.07
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 13208
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 1696
	Voluntary context switches: 23
	Involuntary context switches: 23
	Swaps: 0
	File system inputs: 16
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


`Major (requiring I/O)` = $0$

### 3) Validación a través de `%timeit`

Con un loop de tamaño 5 y 30 repeticiones se logra una medición estable en aproximadamente 69 milisegundos como el mejor resultado de la prueba.

In [128]:
%timeit -n 5 -r 30 PCA_from_SVD(seccion_3,2)

5 loops, best of 30: 69.7 ms per loop


## Medición de uso de CPU

### 1) A través de cProfile

### 2) A través de line_profiler

## Uso de memoria RAM

### 1)A través de `%memit`

### 2) A través de `memory_profiler`

### 3) A través de heapy